# Итоговое задание №2. Прогнозирование конечных свойств новых материалов (композиционных материалов)
На входе имеются данные о начальных свойствах компонентов композиционных материалов (количество связующего, наполнителя, температурный режим отверждения и т.д.). На выходе необходимо спрогнозировать ряд конечных свойств получаемых композиционных материалов. 

Предсказание (из семинара 22): 
* модуль упругости при растяжении
* прочности при растяжении
* cоотношение матрица-наполнитель

Предсказание (из итогового вебинара 26.11.2022):
* модуль упругости при растяжении
* прочности при растяжении

# Разделение данных на обучающую, тестовую и валидационную выборки

In [1]:
import numpy as np
import pandas as pd
import sklearn as sl

In [2]:
df = pd.read_csv('../data/interim/df.csv', sep=';',index_col=0) # Загрузка и создание датафрейма из csv файла
df_encode = pd.read_csv('../data/interim/df_encode.csv', sep=';',index_col=0) # Загрузка и создание датафрейма из csv файла
df_encode_drop = pd.read_csv('../data/interim/df_encode_drop.csv', sep=';',index_col=0) # Загрузка и создание датафрейма из csv файла
df_encode_drop_norm = pd.read_csv('../data/interim/df_encode_drop_norm.csv', sep=';',index_col=0) # Загрузка и создание датафрейма из csv файла

In [ ]:
df_encode_drop_norm.info()

In [37]:
y_df_elastic =  pd.DataFrame(df_encode_drop_norm['Модуль упругости при растяжении, ГПа'],
                            columns=['Модуль упругости при растяжении, ГПа'])
y_df_strength = pd.DataFrame(df_encode_drop_norm['Прочность при растяжении, МПа'],
                            columns=['Прочность при растяжении, МПа'])
y_df_matrix = pd.DataFrame(df_encode_drop_norm['Соотношение матрица-наполнитель'],
                            columns=['Соотношение матрица-наполнитель'])

In [38]:
x_df_elastic = df_encode_drop_norm.loc[:, df_encode_drop_norm.columns != 'Модуль упругости при растяжении, ГПа']
x_df_strength = df_encode_drop_norm.loc[:, df_encode_drop_norm.columns != 'Прочность при растяжении, МПа']
x_df_matrix = df_encode_drop_norm.loc[:, df_encode_drop_norm.columns != 'Соотношение матрица-наполнитель']


Разделение и сохранение датасетов на Train ,test, valid

In [44]:
from sklearn.model_selection import train_test_split

def Divide_2_train_test_val(x_df, y_df, nm_str):
    X_train_val, X_test, y_train_val, y_test = train_test_split(x_df.values,
                            y_df.values, test_size=.3, random_state=17)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val,
                            y_train_val, test_size=.3, random_state=17)

    print(f'Количество объектов в тренировочных данных {nm_str}: {X_train.shape[0]}')
    print(f'Количество объектов в тестовых данных {nm_str}: {X_test.shape[0]}')
    print(f'Количество объектов в валидационных данных {nm_str}: {X_val.shape[0]}') 

    df_save = pd.DataFrame(X_train, columns=x_df.columns)
    df_save.to_csv('../data/interim/X_train_' + nm_str +'.csv', sep=';')

    df_save = pd.DataFrame(X_test, columns=x_df.columns)
    df_save.to_csv('../data/interim/X_test_'+nm_str+'.csv', sep=';')

    df_save = pd.DataFrame(X_val, columns=x_df.columns)
    df_save.to_csv('../data/interim/X_val_'+nm_str+'.csv', sep=';')

    df_save = pd.DataFrame(y_train, columns=y_df.columns)
    df_save.to_csv('../data/interim/y_train_'+nm_str+'.csv', sep=';')

    df_save = pd.DataFrame(y_test, columns=y_df.columns)
    df_save.to_csv('../data/interim/y_test_'+nm_str+'.csv', sep=';')

    df_save = pd.DataFrame(y_val, columns=y_df.columns)
    df_save.to_csv('../data/interim/y_val_'+nm_str+'.csv', sep=';')

In [46]:
Divide_2_train_test_val(x_df_elastic, y_df_elastic, 'elastic')
Divide_2_train_test_val(x_df_strength, y_df_strength, 'strength')
Divide_2_train_test_val(x_df_matrix, y_df_matrix, 'matrix')

Количество объектов в тренировочных данных elastic: 489
Количество объектов в тестовых данных elastic: 300
Количество объектов в валидационных данных elastic: 210
Количество объектов в тренировочных данных strength: 489
Количество объектов в тестовых данных strength: 300
Количество объектов в валидационных данных strength: 210
Количество объектов в тренировочных данных matrix: 489
Количество объектов в тестовых данных matrix: 300
Количество объектов в валидационных данных matrix: 210


y_column_names = ['Модуль упругости при растяжении, ГПа','Прочность при растяжении, МПа','Соотношение матрица-наполнитель']
x_column_names = ['Плотность, кг/м3', 'модуль упругости, ГПа','Количество отвердителя, м.%','Содержание эпоксидных групп,%_2','Температура вспышки, С_2','Поверхностная плотность, г/м2','Потребление смолы, г/м2','Угол нашивки, град','Шаг нашивки','Плотность нашивки']

dfX = df.drop(columns=y_column_names)
dfY = df.drop(columns=x_column_names)